In [ ]:
import tensorflow as tf

def mpii_read_and_decode_single_example(tfrecord):
    '''
    args: tfrecord for the mpii dataset
    returns: parsed example from the tfrecord
    '''
    
    # first construct a queue containing a list of filenames.
    # this lets a user split up there dataset in multiple files to keep
    # size down
    filename_queue = tf.train.string_input_producer([tfrecord],
                                                    num_epochs=None)
    
    # Unlike the TFRecordWriter, the TFRecordReader is symbolic
    reader = tf.TFRecordReader()
    
    # One can read a single serialized example from a filename
    # serialized_example is a Tensor of type string.
    _, serialized_example = reader.read(filename_queue)
    
    # the feature map for the mpii dataset allows us to convert the serialized
    # example back to non-serialized values
    feature_map = {
        'image_jpeg': tf.FixedLenFeature(shape=[], dtype=tf.string),
        'joint_indices': tf.VarLenFeature(dtype=tf.int64),
        'x_joints': tf.VarLenFeature(dtype=tf.float32),
        'y_joints': tf.VarLenFeature(dtype=tf.float32),
        'head_size': tf.FixedLenFeature(shape=[], dtype=tf.float32)
    }
    
    # The serialized example is converted back to actual values.
    # One needs to describe the format of the objects to be returned
    example = tf.parse_single_example(
        serialized_example,
        features=feature_map)
    # now return the converted data
    return example

x = mpii_read_and_decode_single_example('data/train0.tfrecord')
img_jpeg = x['image_jpeg']
img_tensor = tf.image.decode_jpeg(contents=img_jpeg, channels=3)
decoded_img = tf.image.convert_image_dtype(image=img_tensor, dtype=tf.float32)
reshaped_img = tf.reshape(tensor=decoded_img,shape=[380,380,3])

sess = tf.Session()
sess.run(tf.initialize_all_variables())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

x = sess.run(reshaped_img)